In [ ]:
import os
import shutil
import time
import pandas as pd
from tqdm.notebook import tqdm
import re
import file_function as ff
from os.path import join

In [ ]:
path = r'C:\Users\SL\Desktop\test'
ff.moveFilesToRootFolderAndRmDirs(path)

### 파일명 작성 지침 변경에 따른 부산 수정

##### 파일 전체 옮기기

In [ ]:
path = r'D:\부산 정리_v1'
dst_root = r'D:\부산 정리_v2'
ff.copy_all(path, dst_root)

##### '보증인', 집행권원, 강제집행 수정

In [ ]:
path = r'D:\부산 정리_v2'
filelist = os.walk(path)

for root, dirs, files in tqdm(filelist) :
    for f in files :
        new_name = f

        if re.search("판결문|지급명령|이행권고|화해권고", new_name) :
            new_name = re.sub("판결문|지급명령|이행권고|화해권고", "집행권원", new_name)

        elif re.search("타채|결정문", new_name) :
            new_name = re.sub("타채|결정문", "강제집행", new_name)

        elif re.search("채권양도통지서", new_name) :
            new_name = re.sub("채권양도통지서", "양도통지서", new_name)

        if re.search("[_]+보증인", new_name) :
            split_list = re.sub("[_]+보증인", "", new_name).split("_")
            split_list[1] = "보증인" + " "+ split_list[1]

            new_name = "_".join(split_list)
        
        if new_name != f : 
                                        ########
            ff.re_name(join(root,f), join(root, new_name))
                                        ########
    

##### 사건번호 다시 넣기

In [ ]:
v2_dict = ff.fileInfoDict(r"D:\부산 정리_v2")

In [ ]:
len(v2_dict) #{채무자키 : {size : fullpath of files in 부산정리_v2}}

In [ ]:
# 사건번호 있는 파일 목록 작성
p_key = re.compile(r"([\d]{8})[\D]") #match
p_event = re.compile(r"[\D]([\d]{4}[ㄱ-ㅎ가-힣]{1,3}[0-9]+)")
p_extension = re.compile('(jpeg|jpg|bmp|gif|pdf|png|tif|xps)$', re.I)

no_key_error = []
no_file_error = []
result = [] #4740개 나와야


path = r'D:\1.다운로드\__부산 공유폴더_원본'
filelist = os.walk(path)

In [ ]:
index = 0
for root, __dirs__, files in tqdm(filelist) :
    
    for f in files :
        index += 1
        if (p_extension.search(f) != None) : #스캔본인 경우
            
            # [주의] 사건번호에 띄어쓰기 없어야 하므로 파일명에서 띄어쓰기 제거
            nospace_f = re.sub(" ", "", f)

            if p_event.search(nospace_f) != None : # 사건번호 있으면
                if p_key.search(f) != None : # 원본이라 채무자키가 젤 앞에 없는 경우도 많다.
                    
                    key = str(p_key.search(f).group(1)) # str() 유의
                    size = str(os.path.getsize(join(root, f))) # str() 유의
                    event = p_event.search(nospace_f).group(1) #nospace_f, group(1) 유의
                    
                    try :
                        src = v2_dict[key][size] #변경할 파일 풀경로
                        
                        src_dir = os.path.split(src)[0]
                        src_filename = os.path.split(src)[1]
                        src_stem = os.path.splitext(src_filename)[0]
                        src_ext = os.path.splitext(src_filename)[1]
                        
                        # 사건번호 넣기
                        src_split_list = src_stem.split("_")
                        src_split_list.insert(3, event) 
                        
                        dst_filename = "_".join(src_split_list) + src_ext
                        dst = join(src_dir, dst_filename)
                        
                        result.append(ff.re_name(src, dst))

                    except :
                        no_file_error.append([key, size, event, join(root, f)])
                        continue #아래 코드 실행 안 됨
                else :
                    no_key_error.append(join(root, f))
            else : pass
        else : pass

print(index)

In [ ]:
print(result)

##### 양통 차수, 집행권원 재도

In [ ]:
path = r'D:\부산 정리_v2'
filelist = os.walk(path)
result = []

p_재도 = re.compile("_재도$")
p_차수 = re.compile("_([0-9]차)$")

In [ ]:
for root, __dirs__, files in filelist :

    for f in files :

        src = join(root, f)
        stem = os.path.splitext(f)[0]
        ext = os.path.splitext(f)[1]

        if p_재도.search(stem) :
            
            stem = p_재도.sub("", stem)
            split_list = stem.split("_")
            split_list[2]= split_list[2]+" "+"재도"

            dst_filename = "_".join(split_list) + ext
            dst = join(root, dst_filename)
            result.append(ff.re_name(src, dst))
        
        elif p_차수.search(stem) :

            차수 = p_차수.search(stem).group(1)
            stem = p_차수.sub("", stem)
            split_list = stem.split("_")
            split_list[2] = split_list[2]+ " " + 차수

            dst_filename = "_".join(split_list) + ext
            dst = join(root, dst_filename)
            result.append(ff.re_name(src, dst))

print(len(result), "개의 이름 변경")

##### 재산조회, 녹취록 (수작업 함)

### 체크하기

##### >>> 채무자키만

In [ ]:
path = r'D:\부산 정리'
os.chdir(path)
file_list = os.walk(path)

p_key = re.compile(r'[\d]{8}[\D]')
p_notkey = re.compile('[0-9]{9,}')
error = []

for r, d, files in tqdm(file_list) :
    for f in files :
        if f != "Thumbs.db":
            if p_key.match(f) == None :
                error.append(f)
print(error)

##### >>>문서종류만

In [ ]:
p_docu = re.compile('원인서류|채권양도통지서|판결문|지급명령|이행권고|화해권고|타채|결정문|등본|초본|개회|신복|파산|외국인증명')

path = r'D:\부산 정리'
file_list = os.walk(path)
error_short = []
error_docu = []

for r, d, files in tqdm(file_list) :
    for f in files :
        if f != "Thumbs.db":
            splitname = f.split("_")
            if len(splitname)<3 :
                error_short.append(f)
            elif p_docu.search(splitname[2]) == None :
                error_docu.append(f)
print("short :", *error_short, sep="\n")
print("docu :", *error_docu, sep="\n")

### 돌발상황 처리

##### 이름 중간에 들어간 _


In [ ]:
### 등,초본이 초본을 포함해버린다. 이거 수정해야
path = r'D:\부산 정리'

p_docu = re.compile(
    '원인서류|채권양도통지서|판결문|지급명령|이행권고|화해권고|타채|결정문|등본|초본|개회|신복|파산|외국인증명')
p_ = re.compile("_")

os.chdir(path)
file_list = os.walk(path)
fixed = []
count = 0

for r, d, files in tqdm(file_list):
    for f in files:
        if f != "Thumbs.db":
            new_name = ""
            if p_docu.search(f):
                st = p_docu.search(f).start() - 1
                new_name = f[:9] + p_.sub(" ", f[9:st]) + f[st:]  # 앞뒤 언더바 말고 중간에 언더바

                if f != new_name:
                    os.rename(r+"/"+f, r+"/"+new_name)
                    count += 1
                    fixed.append([f, new_name])
                    

print(count, "개의 파일 이름 변경 완료")
print(*fixed, sep="\n")

##### 잘린 외국인 이름 정정

In [ ]:
# 파일명에 보증인 키워드가 있는지만 보고 그거에 맞춰서 ... 

# 채무자조회파일 dict로 불러오기
dict = ff.debtorInfoDict()

path = r'D:\부산 정리'
file_list = os.walk(path)

p_eng = re.compile("[a-zA-Z]+")
p_etc = re.compile("보증인")

count = 0

for root, dir, files in tqdm(file_list) : 
    for f in files : 

        if f != "Thumbs.db":
            new_name = ""
            
            spl_name = f.split("_", maxsplit=2)

            key = spl_name[0]
            debtor_name = spl_name[1]
            others = spl_name[2]

            if p_eng.search(debtor_name) : # 이름에 영어 있는 경우
                if p_etc.search(others) : # 보증인 키워드 있는 경우
                    new_name = key +"_" + str(dict[key].보증인성명)+"_"+ others
                else : # 보증인 키워드 없는 경우
                    new_name = key +"_" + str(dict[key].성명)+"_"+others
                    
                if new_name != f :
                    src = join(root, f)
                    dst = join(root, new_name)
                    ff.re_name(src, dst)
                    count += 1
print(count, "개의 파일명 변경 완료")
                  